In [7]:
import pandas as pd
import numpy as np

In [8]:
listings_data = pd.read_csv('../data/listings.csv.gz', compression='gzip')

In [9]:
listings_data

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,3781,https://www.airbnb.com/rooms/3781,20250315060211,2025-03-15,city scrape,HARBORSIDE-Walk to subway,Fully separate apartment in a two apartment bu...,"Mostly quiet ( no loud music, no crowed sidewa...",https://a0.muscache.com/pictures/24670/b2de044...,4804,...,4.96,4.85,4.88,NaN,f,1,1,0,0,0.22
1,5506,https://www.airbnb.com/rooms/5506,20250315060211,2025-03-15,city scrape,** Fort Hill Inn Private! Minutes to center!**,**THE BEST Value in BOSTON!!*** PRIVATE GUEST ...,"Peaceful, Architecturally interesting, histori...",https://a0.muscache.com/pictures/miso/Hosting-...,8229,...,4.89,4.56,4.76,STR-490093,f,11,11,0,0,0.68
2,6695,https://www.airbnb.com/rooms/6695,20250315060211,2025-03-15,city scrape,"Fort Hill Inn *Sunny* 1 bedroom, condo duplex","Comfortable, Fully Equipped private apartment...","Peaceful, Architecturally interesting, histori...",https://a0.muscache.com/pictures/38ac4797-e7a4...,8229,...,4.95,4.53,4.72,STR-491702,f,11,11,0,0,0.71
3,8789,https://www.airbnb.com/rooms/8789,20250315060211,2025-03-15,previous scrape,Curved Glass Studio/1bd facing Park,This unit is for sale. There will need to be o...,Beacon Hill is a historic neighborhood filled ...,https://a0.muscache.com/pictures/miso/Hosting-...,26988,...,4.97,4.97,4.59,NaN,f,6,6,0,0,0.22
4,10811,https://www.airbnb.com/rooms/10811,20250315060211,2025-03-15,city scrape,"Back Bay Apt Studio-3 blocks to Pru center & ""T""",Stunning Back Bay furnished studio apartment. ...,A one-square mile neighborhood that is arguabl...,https://a0.muscache.com/pictures/45735/27548f7...,38997,...,4.00,5.00,4.67,NaN,f,12,12,0,0,0.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4603,1375656401110940289,https://www.airbnb.com/rooms/1375656401110940289,20250315060211,2025-03-15,city scrape,Fabulous North End 3 BR | 1.5 BA in Little Italy,Newly renovated 3 Bedroom | 1.5 Bath ground-le...,The North End is Boston's oldest residential n...,https://a0.muscache.com/pictures/e41ad5a1-1966...,9410008,...,NaN,NaN,NaN,STR-544876,t,16,16,0,0,NaN
4604,1375662640245506811,https://www.airbnb.com/rooms/1375662640245506811,20250315060211,2025-03-15,city scrape,"Classy, Central Beacon Hill Penthouse - NEW",Enjoy a stylish experience at this centrally-l...,Located in an upscale historic district in dow...,https://a0.muscache.com/pictures/hosting/Hosti...,263259492,...,NaN,NaN,NaN,STR-586938,f,2,2,0,0,NaN
4605,1375681174270903038,https://www.airbnb.com/rooms/1375681174270903038,20250315060211,2025-03-15,city scrape,Luxury 3brm/1bath Duplex in JP,This thoughtfully furnished and well designed ...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,2006852,...,NaN,NaN,NaN,STR-584174,t,21,21,0,0,NaN
4606,1375695500454617370,https://www.airbnb.com/rooms/1375695500454617370,20250315060211,2025-03-15,city scrape,Large 3.5 BR|2 BA Penthouse Duplex,Just updated 3.5 BR | 2 BA condo that features...,Centrally located in Boston's historic Beacon ...,https://a0.muscache.com/pictures/83937472/0272...,9410008,...,NaN,NaN,NaN,STR-544866,t,16,16,0,0,NaN


In [10]:
listings_data.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

In [19]:
listings_columns = [
    "id", "name", "description", "host_id", "listing_url", "neighborhood_overview",
    "picture_url", "latitude", "longitude", "property_type", "room_type", "accommodates",
    "bathrooms","bathrooms_text", "bedrooms", "beds", "amenities", "license", 'review_scores_rating', 'review_scores_accuracy',
    'review_scores_cleanliness', 'review_scores_checkin',
    'review_scores_communication', 'review_scores_location',
    'review_scores_value','number_of_reviews'
]


In [20]:
listings_df = listings_data[listings_columns].copy()
listings_df.rename(columns={'id': 'listing_id',
    'review_scores_rating': 'overall_rating',
    'review_scores_accuracy': 'accuracy_rating',
    'review_scores_cleanliness': 'cleanliness_rating',
    'review_scores_checkin': 'checkin_rating',
    'review_scores_communication': 'communication_rating',
    'review_scores_location': 'location_rating',
    'review_scores_value': 'value_rating'
}, inplace=True)

In [21]:
listings_df

,listing_id,name,description,host_id,listing_url,neighborhood_overview,picture_url,latitude,longitude,property_type,...,amenities,license,overall_rating,accuracy_rating,cleanliness_rating,checkin_rating,communication_rating,location_rating,value_rating,number_of_reviews
0,3781,HARBORSIDE-Walk to subway,Fully separate apartment in a two apartment bu...,4804,https://www.airbnb.com/rooms/3781,"Mostly quiet ( no loud music, no crowed sidewa...",https://a0.muscache.com/pictures/24670/b2de044...,42.364130,-71.029910,Entire rental unit,...,"[""Oven"", ""Extra pillows and blankets"", ""Cookin...",NaN,4.96,5.00,4.96,5.00,4.96,4.85,4.88,26
1,5506,** Fort Hill Inn Private! Minutes to center!**,**THE BEST Value in BOSTON!!*** PRIVATE GUEST ...,8229,https://www.airbnb.com/rooms/5506,"Peaceful, Architecturally interesting, histori...",https://a0.muscache.com/pictures/miso/Hosting-...,42.328440,-71.095810,Entire guest suite,...,"[""Extra pillows and blankets"", ""Hair dryer"", ""...",STR-490093,4.81,4.88,4.91,4.95,4.89,4.56,4.76,132
2,6695,"Fort Hill Inn *Sunny* 1 bedroom, condo duplex","Comfortable, Fully Equipped private apartment...",8229,https://www.airbnb.com/rooms/6695,"Peaceful, Architecturally interesting, histori...",https://a0.muscache.com/pictures/38ac4797-e7a4...,42.328020,-71.093870,Entire condo,...,"[""Oven"", ""Cooking basics"", ""Hair dryer"", ""Kitc...",STR-491702,4.81,4.83,4.86,4.91,4.95,4.53,4.72,135
3,8789,Curved Glass Studio/1bd facing Park,This unit is for sale. There will need to be o...,26988,https://www.airbnb.com/rooms/8789,Beacon Hill is a historic neighborhood filled ...,https://a0.muscache.com/pictures/miso/Hosting-...,42.358670,-71.063070,Entire rental unit,...,"[""Extra pillows and blankets"", ""Cooking basics...",NaN,4.69,4.69,4.55,4.93,4.97,4.97,4.59,29
4,10811,"Back Bay Apt Studio-3 blocks to Pru center & ""T""",Stunning Back Bay furnished studio apartment. ...,38997,https://www.airbnb.com/rooms/10811,A one-square mile neighborhood that is arguabl...,https://a0.muscache.com/pictures/45735/27548f7...,42.351730,-71.086850,Entire rental unit,...,"[""Hair dryer"", ""Kitchen"", ""Essentials"", ""Pets ...",NaN,4.33,4.44,4.67,4.56,4.00,5.00,4.67,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4603,1375656401110940289,Fabulous North End 3 BR | 1.5 BA in Little Italy,Newly renovated 3 Bedroom | 1.5 Bath ground-le...,9410008,https://www.airbnb.com/rooms/1375656401110940289,The North End is Boston's oldest residential n...,https://a0.muscache.com/pictures/e41ad5a1-1966...,42.367523,-71.052688,Entire rental unit,...,"[""Oven"", ""Extra pillows and blankets"", ""Cookin...",STR-544876,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4604,1375662640245506811,"Classy, Central Beacon Hill Penthouse - NEW",Enjoy a stylish experience at this centrally-l...,263259492,https://www.airbnb.com/rooms/1375662640245506811,Located in an upscale historic district in dow...,https://a0.muscache.com/pictures/hosting/Hosti...,42.357720,-71.071090,Entire rental unit,...,"[""Oven"", ""Extra pillows and blankets"", ""Hair d...",STR-586938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4605,1375681174270903038,Luxury 3brm/1bath Duplex in JP,This thoughtfully furnished and well designed ...,2006852,https://www.airbnb.com/rooms/1375681174270903038,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,42.318576,-71.106661,Entire rental unit,...,"[""Oven"", ""Extra pillows and blankets"", ""Cookin...",STR-584174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4606,1375695500454617370,Large 3.5 BR|2 BA Penthouse Duplex,Just updated 3.5 BR | 2 BA condo that features...,9410008,https://www.airbnb.com/rooms/1375695500454617370,Centrally located in Boston's historic Beacon ...,https://a0.muscache.com/pictures/83937472/0272...,42.360268,-71.069963,Entire rental unit,...,"[""Oven"", ""Extra pillows and blankets"", ""Cookin...",STR-544866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
# Filling in missing values
listings_df['description'] = listings_df['description'].fillna('No description given')
listings_df['neighborhood_overview'] = listings_df['neighborhood_overview'].fillna('No neighborhood overview given')
listings_df['license'] = listings_df['license'].fillna('No license information')

listings_df['bathrooms'] = np.where(
    listings_df['bathrooms'].isnull(),
    listings_df['bathrooms_text'].str.extract(r'(\d+\.?\d*)').astype(float),
    listings_df['bathrooms']
)

listings_df['bathrooms'] = listings_df['bathrooms'].fillna(1) 
listings_df['bedrooms'] = listings_df['bedrooms'].fillna(1)  
listings_df['beds'] = listings_df['beds'].fillna(1)

listings_df['bathroom_type'] = np.where(
    listings_df['bathrooms_text'].str.contains('shared', case=False, na=False),
    'shared',
    'private'
)

listings_df['amenities'] = listings_df['amenities'].fillna('No amenities listed')

listings_df.drop(columns=['bathrooms_text'], inplace=True)

In [23]:
listings_df

,listing_id,name,description,host_id,listing_url,neighborhood_overview,picture_url,latitude,longitude,property_type,...,license,overall_rating,accuracy_rating,cleanliness_rating,checkin_rating,communication_rating,location_rating,value_rating,number_of_reviews,bathroom_type
0,3781,HARBORSIDE-Walk to subway,Fully separate apartment in a two apartment bu...,4804,https://www.airbnb.com/rooms/3781,"Mostly quiet ( no loud music, no crowed sidewa...",https://a0.muscache.com/pictures/24670/b2de044...,42.364130,-71.029910,Entire rental unit,...,No license information,4.96,5.00,4.96,5.00,4.96,4.85,4.88,26,private
1,5506,** Fort Hill Inn Private! Minutes to center!**,**THE BEST Value in BOSTON!!*** PRIVATE GUEST ...,8229,https://www.airbnb.com/rooms/5506,"Peaceful, Architecturally interesting, histori...",https://a0.muscache.com/pictures/miso/Hosting-...,42.328440,-71.095810,Entire guest suite,...,STR-490093,4.81,4.88,4.91,4.95,4.89,4.56,4.76,132,private
2,6695,"Fort Hill Inn *Sunny* 1 bedroom, condo duplex","Comfortable, Fully Equipped private apartment...",8229,https://www.airbnb.com/rooms/6695,"Peaceful, Architecturally interesting, histori...",https://a0.muscache.com/pictures/38ac4797-e7a4...,42.328020,-71.093870,Entire condo,...,STR-491702,4.81,4.83,4.86,4.91,4.95,4.53,4.72,135,private
3,8789,Curved Glass Studio/1bd facing Park,This unit is for sale. There will need to be o...,26988,https://www.airbnb.com/rooms/8789,Beacon Hill is a historic neighborhood filled ...,https://a0.muscache.com/pictures/miso/Hosting-...,42.358670,-71.063070,Entire rental unit,...,No license information,4.69,4.69,4.55,4.93,4.97,4.97,4.59,29,private
4,10811,"Back Bay Apt Studio-3 blocks to Pru center & ""T""",Stunning Back Bay furnished studio apartment. ...,38997,https://www.airbnb.com/rooms/10811,A one-square mile neighborhood that is arguabl...,https://a0.muscache.com/pictures/45735/27548f7...,42.351730,-71.086850,Entire rental unit,...,No license information,4.33,4.44,4.67,4.56,4.00,5.00,4.67,9,private
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4603,1375656401110940289,Fabulous North End 3 BR | 1.5 BA in Little Italy,Newly renovated 3 Bedroom | 1.5 Bath ground-le...,9410008,https://www.airbnb.com/rooms/1375656401110940289,The North End is Boston's oldest residential n...,https://a0.muscache.com/pictures/e41ad5a1-1966...,42.367523,-71.052688,Entire rental unit,...,STR-544876,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,private
4604,1375662640245506811,"Classy, Central Beacon Hill Penthouse - NEW",Enjoy a stylish experience at this centrally-l...,263259492,https://www.airbnb.com/rooms/1375662640245506811,Located in an upscale historic district in dow...,https://a0.muscache.com/pictures/hosting/Hosti...,42.357720,-71.071090,Entire rental unit,...,STR-586938,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,private
4605,1375681174270903038,Luxury 3brm/1bath Duplex in JP,This thoughtfully furnished and well designed ...,2006852,https://www.airbnb.com/rooms/1375681174270903038,No neighborhood overview given,https://a0.muscache.com/pictures/miso/Hosting-...,42.318576,-71.106661,Entire rental unit,...,STR-584174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,private
4606,1375695500454617370,Large 3.5 BR|2 BA Penthouse Duplex,Just updated 3.5 BR | 2 BA condo that features...,9410008,https://www.airbnb.com/rooms/1375695500454617370,Centrally located in Boston's historic Beacon ...,https://a0.muscache.com/pictures/83937472/0272...,42.360268,-71.069963,Entire rental unit,...,STR-544866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,private


In [4]:
calendar_data = pd.read_csv('../data/calendar.csv.gz', compression='gzip')

In [5]:
calendar_data

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,3781,2025-03-15,f,$125.00,NaN,29,1125
1,3781,2025-03-16,t,$125.00,NaN,29,1125
2,3781,2025-03-17,t,$125.00,NaN,29,1125
3,3781,2025-03-18,t,$125.00,NaN,29,1125
4,3781,2025-03-19,t,$125.00,NaN,29,1125
...,...,...,...,...,...,...,...
1680163,2215762,2026-03-10,f,$180.00,NaN,3,1125
1680164,2215762,2026-03-11,f,$180.00,NaN,3,1125
1680165,2215762,2026-03-12,f,$180.00,NaN,3,1125
1680166,2215762,2026-03-13,f,$180.00,NaN,3,1125
